In [11]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from lxml import html

In [12]:
comp_vc_url = 'https://vc.ru/subs/companies'

In [13]:
r = requests.get(comp_vc_url)

In [14]:
soup = BeautifulSoup(r.text, 'lxml')

In [15]:
companies_vc = []
status_vc = ['подписаться', 'отписаться', 'подписан', 'подписки', 'компании', '']
for child in soup.recursiveChildGenerator():
    text = child.text.strip().lower()
    if child.name == 'span' and (not text in status_vc) and (not 'ваканс' in text):
        companies_vc.append(child.text.strip().lower())

In [17]:
len(companies_vc)

476

In [18]:
with open('companies_vc.txt', 'w') as f:
    f.write('\n'.join(companies_vc))

In [9]:
#with open('companies_vc.txt', 'r') as f:
#    data = f.read()

In [8]:
def find_companies_from_habr(string):
    tree = html.fromstring(string)
    elements = tree.findall(".//a[@class='title'][@target='blank']")
    companies_habr = [elem.text.lower() for elem in elements]
    return companies_habr, len(companies_habr)

In [9]:
#### Переписать with exception
from time import sleep
comp_habr_url = 'https://career.habr.com/companies?'
i = 1
companies_habr = []
pbar = tqdm(total=10000)
len_delta = 1
while r.ok == True or len_delta != 0:
    if i == 1:
        page_url = comp_habr_url
    else:
        page_url = comp_habr_url + 'page=' + str(i)      
    try:
        r = requests.get(page_url)
        delta, len_delta = find_companies_from_habr(r.text)
        if r.ok != True or len_delta == 0:
            break
        else:
            companies_habr += delta.copy()
            i += 1
            pbar.update(1)
            #print(companies_habr)
        
    except requests.exceptions.ConnectionError:
        sleep(60)
        pass
        
pbar.close()

 10%|███▋                                | 1010/10000 [22:11<3:17:34,  1.32s/it]


In [10]:
with open('companies_habr.txt', 'w') as f:
    f.write('\n'.join(companies_habr))